# 3D MPS

> **Note** This notebook is inspired by the notebook [ex_deesse_01_basics.ipynb](https://github.com/randlab/geone/blob/master/examples/ex_deesse_01_basics.ipynb), by Julien Straubhaar.

In this notebook we illustrate how to perform MPS in 3D.

Import the required modules.

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import os
import pyevtk.hl as ph

# import package 'geone'
import geone as gn

## 3D Training image
Load from the file 'ti_3d.txt', and print out some useful information.

In [2]:
ti3d = gn.img.readImageTxt('ti_3d.txt')
ti3d

*** Img object ***
name = 'ti_3d.txt'
(nx, ny, nz) = (100, 90, 80) # number of cells along each axis
(sx, sy, sz) = (1.0, 1.0, 1.0) # cell size (spacing) along each axis
(ox, oy, oz) = (0.5, 0.5, 0.5) # origin (coordinates of bottom-lower-left corner)
nv = 1  # number of variable(s)
varname = ['data']
val: (1, 80, 90, 100)-array
*****

### Saving the 3D TI as VTI file

To visualize the input TI, we can save it as a VTI file to be read with Paraview. You can also visualize it directly in a Python notebook (for more on this, please refer to the script [ex_deesse_01_basics.ipynb](https://github.com/randlab/geone/blob/master/examples/ex_deesse_01_basics.ipynb))

In [3]:
origin = (ti3d.ox, ti3d.oy, ti3d.oz)
spacing = (ti3d.sx, ti3d.sy, ti3d.sz)
cellData = {} # An empty dictionary
for i in range(ti3d.nv):
    # Fill the dictionary with keys as variable names, and content as the input TI
    cellData[ti3d.varname[i]] = ti3d.val[i,:,:,:]
# Save the VTI file
ph.imageToVTK("./ti3d", cellData=cellData, origin=origin, spacing=spacing) # No need to define explicitly the file extension VTI

'/home/alex/workspace/dev/binder/gmg/notebooks/simulation/ti3d.vti'

## Collect some useful info

You can also collect the unique values contained in the TI.

In [4]:
facies = ti3d.get_unique()
facies

array([1., 2., 3.])

## 3D Simulation grid

Define the simulation grid (number of cells in each direction, cell unit, origin). Note that the size of the cells is the same as the TI.

In [5]:
nx, ny, nz = 60, 60, 60          # number of cells
sx, sy, sz = ti3d.sx, ti3d.sy, ti3d.sz # cell unit
ox, oy, oz = 0.0, 0.0, 0.0       # origin (corner of the "first" grid cell)

origin = (ox, oy, oz)
spacing = (sx, sy, sz)

### Hard data (point set)

Define some conditioning data.

In [6]:
npt = 3 # number of points
nv = 4  # number of variables including x, y, z coordinates
varname = ['x', 'y', 'z', 'code'] # list of variable names
v = np.array([
    [ 15.5,  45.5, 50.5, 1], # x, y, z, code: 1st point
    [ 47.5,  10.5, 48.5, 2], # ...
    [ 27.5,  28.5,  5.5, 3]
    ]).T # variable values: (nv, npt)-array
hd = gn.img.PointSet(npt=npt, nv=nv, varname=varname, val=v)

We can save these values in a VTI file image with the same size as the simulation grid, made of NaN values everywhere, exept in the defined points.

In [7]:
cellData = {}
cellData["code"] = np.nan*np.ones((nz, ny, nx))
# Add the hard data to the NaN image
for ii in range(3):
    i = int(v[0][ii]/sx - 0.5*sx - ox)
    j = int(v[1][ii]/sy - 0.5*sy - oy)
    k = int(v[2][ii]/sz - 0.5*sz - oz)
    cellData["code"][k,j,i] = v[3][ii]
# Save the VTI file
ph.imageToVTK("./hd", cellData=cellData, origin=origin, spacing=spacing) # No need to define explicitly the file extension VTI

'/home/alex/workspace/dev/binder/gmg/notebooks/simulation/hd.vti'

## 3D MPS simulation

First of all, define some input parameters.

In [8]:
# Set input for deesse
pyrGenParams = gn.deesseinterface.PyramidGeneralParameters(
    npyramidLevel=2,
    kx=[2, 2], ky=[2, 2], kz=[2, 2]
)

pyrParams = gn.deesseinterface.PyramidParameters(
    nlevel=2,
    pyramidType='categorical_auto'
)

nreal = 3

deesse_input = gn.deesseinterface.DeesseInput(
    nx=nx, ny=ny, nz=nz,
    sx=sx, sy=sy, sz=sz,
    ox=ox, oy=oy, oz=oz,
    nv=1, varname='code',
    TI=ti3d,
    dataPointSet=hd,
    nneighboringNode=24,
    distanceThreshold=0.02,
    maxScanFraction=0.02,
    pyramidGeneralParameters=pyrGenParams,
    pyramidParameters=pyrParams,
    npostProcessingPathMax=1,
    seed=444,
    nrealization=nreal)

Then, run the DS algorithm

In [9]:
deesse_output = gn.deesseinterface.deesseRun(deesse_input, nthreads=4)

DeeSse running... [VERSION 3.2 / BUILD NUMBER 20230914 / OpenMP 11 thread(s)]
DeeSse run complete


## Retrieve and save the results to a VTI file

In [11]:
# Retrieve the realization
sim = deesse_output['sim']
# Empy dictionary to contain the results
cellData = {}
for i in range(sim.size):
    # Loop on all the realizations
    cellData[sim[i].varname[0]] = sim[i].val[0,:,:,:]
# Save the results into a VTI file
ph.imageToVTK("./mps03_3D_sim", cellData=cellData, origin=origin, spacing=spacing) # No need to define explicitly the file extension VTI

'/home/alex/workspace/dev/binder/gmg/notebooks/simulation/mps03_3D_sim.vti'